In [42]:
import json
import pandas as pd
import numpy as np
import warnings
import requests
from tqdm import tqdm
warnings.filterwarnings('ignore')

In [43]:
with open('../matches_info/matches_info0-26000', encoding='utf-8') as file:
    matches_info = json.load(file)

FileNotFoundError: [Errno 2] No such file or directory: '../matches_info/matches_info0-26000'

In [ ]:
with open('../matches_info/new_add', encoding='utf-8') as file:
    new_add = json.load(file)

In [ ]:
matches_id = []
with open('../matches_info/matches_info0-26000', encoding='utf-8') as file:
    matches_info = json.load(file)
matches_relevante = []
for match in matches_info:
    if match['start_time'] > 1518421196:
        matches_relevante.append(match)
        matches_id.append(match['match_id'])

In [ ]:
for match in new_add:
    if match['start_time'] > 1518421196:
        if match['match_id'] not in matches_id: 
            matches_relevante.append(match)
            matches_id.append(match['match_id'])

In [ ]:
with open('../duration_info/matches_after_2018.json', 'w') as file:
    json.dump(matches_relevante, file)

In [ ]:
with open('../duration_info/matches_after_2018.json') as file:
    matches_info = json.load(file)

In [44]:
data = []
for match in matches_info:
    try:
            game = {}
            game['match_id'] = match['match_id']
            game['r_victory'] = match['radiant_win']
            game['d_team_id'] = match['dire_team_id']
            game['r_team_id'] = match['radiant_team_id']
            game['date'] = match['start_time']


            accounts_id = {}
            for i, player in enumerate(match['players']):
                accounts_id[player['hero_id']] = player['account_id']

            
            r, d = 0, 0
            for pick in match['picks_bans']:

                if pick['is_pick'] == True and pick['team']==0:
                    r += 1
                    key = f'r_{r}'
                    hero_id = f'r_{r}_player_id'
                    game[key] = pick['hero_id']
                    game[hero_id] = accounts_id[pick['hero_id']]

                if pick['is_pick'] == True and pick['team']==1:
                    d += 1
                    key = f'd_{d}'
                    hero_id = f'd_{d}_player_id'
                    game[key] = pick['hero_id']
                    game[hero_id] = accounts_id[pick['hero_id']]
                    
            data.append(game)
    except:KeyError
data = pd.DataFrame(data, columns=list(game.keys()))

In [45]:
data = pd.DataFrame(data, columns=list(game.keys()))

In [47]:
%%time
data['r_team_players'] = 0
data['d_team_players'] = 0
r = [f'r_{i}_player_id' for i in range(1, 6)]
d = [f'd_{i}_player_id' for i in range(1, 6)]
for index in range(data.shape[0]):
    data.loc[index, 'r_team_players'] = ''.join(map(str, sorted(data.loc[index, r].values)))
    data.loc[index, 'd_team_players'] = ''.join(map(str, sorted(data.loc[index, d].values)))

Wall time: 55.7 s


In [48]:
teams = list(set(list(data['r_team_players'].values) + list(data['d_team_players'].values)))

In [49]:
result = []
for first in tqdm(teams):
    count = 0
    count += data.loc[(data['r_team_players'] == first), 'r_victory'].count()
    count += data.loc[(data['d_team_players'] == first), 'r_victory'].count()
    if count > 30:
        result.extend(list(data.loc[(data['r_team_players'] == first), 'match_id'].values))
        result.extend(list(data.loc[(data['d_team_players'] == first), 'match_id'].values))

100%|██████████| 1619/1619 [00:11<00:00, 139.57it/s]


In [50]:
result = list(map(str, set(result)))
with open('../duration_info/matches_id.json', 'w', encoding='utf-8') as file:
    json.dump(result, file)

In [51]:
with open('../duration_info/matches_id.json', encoding='utf-8') as file:
    matches_id = json.load(file)
five_mens_res = []
matches_relevante = []
for match in matches_info:
    if match['start_time'] > 1518421196 and str(match['match_id']) in matches_id:
        matches_relevante.append(match)

In [52]:
with open('../duration_info/train.json', 'w') as file:
    json.dump(matches_relevante, file)

In [53]:
orgs_mean = {}
for match in matches_relevante:
    if match['radiant_team_id'] not in orgs_mean:
        orgs_mean[match['radiant_team_id']] = {'wins':0, 'duration':0, 'matches': 0, 'score_mean':0, 'deaths_mean':0, 
                                               'assists':0, 'last_hits':0, 'denies':0, 'gold_per_min':0,
                                               'xp_per_min':0, 'versus_orgs': {}}

    if match['dire_team_id'] not in orgs_mean[match['radiant_team_id']]['versus_orgs']:
        orgs_mean[match['radiant_team_id']]['versus_orgs'][match['dire_team_id']] = {'wins':0, 'duration':0, 'matches': 0,
                                                                                     'score_mean':0, 'deaths_mean':0,
                                                                                     'r_assists':0, 'r_last_hits':0, 
                                                                                     'r_denies':0, 'r_gold_per_min':0,
                                                                                     'r_xp_per_min':0, 'd_assists':0, 
                                                                                     'd_last_hits':0, 'd_denies':0, 
                                                                                     'd_gold_per_min':0, 'd_xp_per_min':0}
    orgs_mean[match['radiant_team_id']]['duration'] += match['duration']
    if match['radiant_win'] == True:
        orgs_mean[match['radiant_team_id']]['wins'] += 1
    orgs_mean[match['radiant_team_id']]['matches'] += 1
    orgs_mean[match['radiant_team_id']]['score_mean'] += match['radiant_score']
    orgs_mean[match['radiant_team_id']]['deaths_mean'] += match['dire_score']

    orgs_mean[match['radiant_team_id']]['versus_orgs'][match['dire_team_id']]['duration'] += match['duration']
    if match['radiant_win'] == True:
        orgs_mean[match['radiant_team_id']]['versus_orgs'][match['dire_team_id']]['wins'] += 1
    orgs_mean[match['radiant_team_id']]['versus_orgs'][match['dire_team_id']]['matches'] += 1
    orgs_mean[match['radiant_team_id']]['versus_orgs'][match['dire_team_id']]['score_mean'] += match['radiant_score']
    orgs_mean[match['radiant_team_id']]['versus_orgs'][match['dire_team_id']]['deaths_mean'] += match['dire_score']



    if match['dire_team_id'] not in orgs_mean:
        orgs_mean[match['dire_team_id']] = {'wins':0, 'duration':0, 'matches': 0, 'score_mean':0, 'deaths_mean':0, 
                                               'assists':0, 'last_hits':0, 'denies':0, 'gold_per_min':0,
                                               'xp_per_min':0, 'versus_orgs': {}}
    if match['radiant_team_id'] not in orgs_mean[match['dire_team_id']]['versus_orgs']:
        orgs_mean[match['dire_team_id']]['versus_orgs'][match['radiant_team_id']] = {'wins':0, 'duration':0, 'matches': 0,
                                                                                     'score_mean':0, 'deaths_mean':0,
                                                                                     'r_assists':0, 'r_last_hits':0, 
                                                                                     'r_denies':0, 'r_gold_per_min':0,
                                                                                     'r_xp_per_min':0, 'd_assists':0, 
                                                                                     'd_last_hits':0, 'd_denies':0, 
                                                                                     'd_gold_per_min':0, 'd_xp_per_min':0}
    orgs_mean[match['dire_team_id']]['duration'] += match['duration']
    orgs_mean[match['dire_team_id']]['matches'] += 1
    if match['radiant_win'] == False:
        orgs_mean[match['dire_team_id']]['wins'] += 1
    orgs_mean[match['dire_team_id']]['score_mean'] += match['dire_score']
    orgs_mean[match['dire_team_id']]['deaths_mean'] += match['radiant_score']

    orgs_mean[match['dire_team_id']]['versus_orgs'][match['radiant_team_id']]['duration'] += match['duration']
    if match['radiant_win'] == False:
        orgs_mean[match['dire_team_id']]['versus_orgs'][match['radiant_team_id']]['wins'] += 1
    orgs_mean[match['dire_team_id']]['versus_orgs'][match['radiant_team_id']]['matches'] += 1
    orgs_mean[match['dire_team_id']]['versus_orgs'][match['radiant_team_id']]['score_mean'] += match['dire_score']
    orgs_mean[match['dire_team_id']]['versus_orgs'][match['radiant_team_id']]['deaths_mean'] += match['radiant_score']



    for i, player in enumerate(match['players']):
        if i < 5:
            orgs_mean[match['radiant_team_id']]['assists'] += player['assists']
            orgs_mean[match['radiant_team_id']]['last_hits'] += player['last_hits']
            orgs_mean[match['radiant_team_id']]['denies'] += player['denies']
            orgs_mean[match['radiant_team_id']]['gold_per_min'] += player['gold_per_min']
            orgs_mean[match['radiant_team_id']]['xp_per_min'] += player['xp_per_min']

            orgs_mean[match['radiant_team_id']]['versus_orgs'][match['dire_team_id']]['r_assists'] += player['assists']
            orgs_mean[match['radiant_team_id']]['versus_orgs'][match['dire_team_id']]['r_last_hits'] += player['last_hits']
            orgs_mean[match['radiant_team_id']]['versus_orgs'][match['dire_team_id']]['r_denies'] += player['denies']
            orgs_mean[match['radiant_team_id']]['versus_orgs'][match['dire_team_id']]['r_gold_per_min'] += player['gold_per_min']
            orgs_mean[match['radiant_team_id']]['versus_orgs'][match['dire_team_id']]['r_xp_per_min'] += player['xp_per_min']
            
            orgs_mean[match['radiant_team_id']]['versus_orgs'][match['dire_team_id']]['d_assists'] += player['assists']
            orgs_mean[match['radiant_team_id']]['versus_orgs'][match['dire_team_id']]['d_last_hits'] += player['last_hits']
            orgs_mean[match['radiant_team_id']]['versus_orgs'][match['dire_team_id']]['d_denies'] += player['denies']
            orgs_mean[match['radiant_team_id']]['versus_orgs'][match['dire_team_id']]['d_gold_per_min'] += player['gold_per_min']
            orgs_mean[match['radiant_team_id']]['versus_orgs'][match['dire_team_id']]['d_xp_per_min'] += player['xp_per_min']

        else:
            orgs_mean[match['dire_team_id']]['assists'] += player['assists']
            orgs_mean[match['dire_team_id']]['last_hits'] += player['last_hits']
            orgs_mean[match['dire_team_id']]['denies'] += player['denies']
            orgs_mean[match['dire_team_id']]['gold_per_min'] += player['gold_per_min']
            orgs_mean[match['dire_team_id']]['xp_per_min'] += player['xp_per_min']

            orgs_mean[match['dire_team_id']]['versus_orgs'][match['radiant_team_id']]['d_assists'] += player['assists']
            orgs_mean[match['dire_team_id']]['versus_orgs'][match['radiant_team_id']]['d_last_hits'] += player['last_hits']
            orgs_mean[match['dire_team_id']]['versus_orgs'][match['radiant_team_id']]['d_denies'] += player['denies']
            orgs_mean[match['dire_team_id']]['versus_orgs'][match['radiant_team_id']]['d_gold_per_min'] += player['gold_per_min']
            orgs_mean[match['dire_team_id']]['versus_orgs'][match['radiant_team_id']]['d_xp_per_min'] += player['xp_per_min']
            
            orgs_mean[match['dire_team_id']]['versus_orgs'][match['radiant_team_id']]['r_assists'] += player['assists']
            orgs_mean[match['dire_team_id']]['versus_orgs'][match['radiant_team_id']]['r_last_hits'] += player['last_hits']
            orgs_mean[match['dire_team_id']]['versus_orgs'][match['radiant_team_id']]['r_denies'] += player['denies']
            orgs_mean[match['dire_team_id']]['versus_orgs'][match['radiant_team_id']]['r_gold_per_min'] += player['gold_per_min']
            orgs_mean[match['dire_team_id']]['versus_orgs'][match['radiant_team_id']]['r_xp_per_min'] += player['xp_per_min']
for org in orgs_mean:
    for stat in orgs_mean[org]:
        if stat != 'matches' and stat != 'versus_orgs':
            orgs_mean[org][stat] /= orgs_mean[org]['matches']
            orgs_mean[org][stat] = round(orgs_mean[org][stat], 2)
    for versus in orgs_mean[org]['versus_orgs']:
        for stat in orgs_mean[org]['versus_orgs'][versus]:
            if stat != 'matches':
                orgs_mean[org]['versus_orgs'][versus][stat] /= orgs_mean[org]['versus_orgs'][versus]['matches']
                if stat == 'wins':
                    orgs_mean[org]['versus_orgs'][versus][stat] = round(orgs_mean[org]['versus_orgs'][versus][stat], 2)
                else:
                    orgs_mean[org]['versus_orgs'][versus][stat] = int(orgs_mean[org]['versus_orgs'][versus][stat])
with open('../duration_info/orgs_mean.json', 'w') as file:
    json.dump(orgs_mean, file)

In [54]:
players_mean = {}
for match in matches_relevante:
    for i, player in enumerate(match['players']):
        if player['account_id'] not in players_mean:
            players_mean[player['account_id']] = {'wins':0, 'duration':0, 'matches': 0, 'score_mean':0, 'deaths_mean':0, 
                                                   'assists':0, 'last_hits':0, 'denies':0, 'gold_per_min':0,
                                                   'xp_per_min':0, 'heroes': {}}
            
        if player['hero_id'] not in  players_mean[player['account_id']]['heroes']:
            players_mean[player['account_id']]['heroes'][player['hero_id']] = {'wins':0, 'duration':0, 'matches': 0, 'score_mean':0, 'deaths_mean':0, 
                                                   'assists':0, 'last_hits':0, 'denies':0, 'gold_per_min':0,
                                                   'xp_per_min':0}
            
        players_mean[player['account_id']]['duration'] += match['duration']
        players_mean[player['account_id']]['matches'] += 1
        players_mean[player['account_id']]['score_mean'] += player['kills']
        players_mean[player['account_id']]['deaths_mean'] += player['deaths']
        players_mean[player['account_id']]['assists'] += player['assists']
        players_mean[player['account_id']]['last_hits'] += player['last_hits']
        players_mean[player['account_id']]['denies'] += player['denies']
        players_mean[player['account_id']]['gold_per_min'] += player['gold_per_min']
        players_mean[player['account_id']]['xp_per_min'] += player['xp_per_min']
        
        if i < 5 and match['radiant_win'] == True:
            players_mean[player['account_id']]['wins'] += 1
            players_mean[player['account_id']]['heroes'][player['hero_id']]['wins'] += 1
            
        if i > 4 and match['radiant_win'] == False:
            players_mean[player['account_id']]['wins'] += 1
            players_mean[player['account_id']]['heroes'][player['hero_id']]['wins'] += 1
        
        
        players_mean[player['account_id']]['heroes'][player['hero_id']]['duration'] += match['duration']
        players_mean[player['account_id']]['heroes'][player['hero_id']]['matches'] += 1
        players_mean[player['account_id']]['heroes'][player['hero_id']]['score_mean'] += player['kills']
        players_mean[player['account_id']]['heroes'][player['hero_id']]['deaths_mean'] += player['deaths']
        players_mean[player['account_id']]['heroes'][player['hero_id']]['assists'] += player['assists']
        players_mean[player['account_id']]['heroes'][player['hero_id']]['last_hits'] += player['last_hits']
        players_mean[player['account_id']]['heroes'][player['hero_id']]['denies'] += player['denies']
        players_mean[player['account_id']]['heroes'][player['hero_id']]['gold_per_min'] += player['gold_per_min']
        players_mean[player['account_id']]['heroes'][player['hero_id']]['xp_per_min'] += player['xp_per_min']
for player in players_mean:
    for stat in players_mean[player]:
        if stat != 'heroes' and stat != 'matches':
            players_mean[player][stat] /= players_mean[player]['matches']
            players_mean[player][stat] = round(players_mean[player][stat], 2)
    for hero in players_mean[player]['heroes']:
        for stat in players_mean[player]['heroes'][hero]:
            if stat != 'matches':
                players_mean[player]['heroes'][hero][stat] /= players_mean[player]['heroes'][hero]['matches']
                players_mean[player]['heroes'][hero][stat] = round(players_mean[player]['heroes'][hero][stat], 2)
with open('../duration_info/players_mean.json', 'w') as file:
    json.dump(players_mean, file)

In [55]:
heroes_mean = {}
for match in matches_relevante:
    for i, player in enumerate(match['players']):
        if player['hero_id'] not in heroes_mean:
            heroes_mean[player['hero_id']] = {'wins':0, 'duration':0, 'matches': 0, 'score_mean':0, 'deaths_mean':0, 
                                                   'assists':0, 'last_hits':0, 'denies':0, 'gold_per_min':0,
                                                   'xp_per_min':0}
            
        if i < 5 and match['radiant_win'] == True:
            heroes_mean[player['hero_id']]['wins'] += 1
            
        if i > 4 and match['radiant_win'] == False:
            heroes_mean[player['hero_id']]['wins'] += 1
        
        heroes_mean[player['hero_id']]['duration'] += match['duration']
        heroes_mean[player['hero_id']]['matches'] += 1
        heroes_mean[player['hero_id']]['score_mean'] += player['kills']
        heroes_mean[player['hero_id']]['deaths_mean'] += player['deaths']
        heroes_mean[player['hero_id']]['assists'] += player['assists']
        heroes_mean[player['hero_id']]['last_hits'] += player['last_hits']
        heroes_mean[player['hero_id']]['denies'] += player['denies']
        heroes_mean[player['hero_id']]['gold_per_min'] += player['gold_per_min']
        heroes_mean[player['hero_id']]['xp_per_min'] += player['xp_per_min']
for hero in heroes_mean:
    for stat in heroes_mean[hero]:
        if stat != 'matches':
            heroes_mean[hero][stat] /= heroes_mean[hero]['matches']
            heroes_mean[hero][stat] = round(heroes_mean[hero][stat], 2)
with open('../duration_info/heroes_mean.json', 'w') as file:
    json.dump(heroes_mean, file)

In [56]:
five_man_mean = {}
for match in matches_relevante:
    radiant = []
    dire = []
    r_wins = 0
    r_assists = 0
    r_last_hits = 0
    r_denies = 0
    r_gpm = 0
    r_xpm = 0
    d_wins = 0
    d_assists = 0
    d_last_hits = 0
    d_denies = 0
    d_gpm = 0
    d_xpm = 0
    for i, player in enumerate(match['players']):
        if i < 5:
            r_assists += player['assists']
            r_last_hits += player['last_hits']
            r_denies += player['denies']
            r_gpm += player['gold_per_min']
            r_xpm += player['xp_per_min']
            if match['radiant_win'] == True:
                r_wins += 1
            radiant.append(player['account_id'])
        else:
            d_assists += player['assists']
            d_last_hits += player['last_hits']
            d_denies += player['denies']
            d_gpm += player['gold_per_min']
            d_xpm += player['xp_per_min']
            if match['radiant_win'] == False:
                d_wins += 1
            dire.append(player['account_id'])
    radiant = ''.join(map(str, sorted(radiant)))
    dire = ''.join(map(str, sorted(dire)))
    
    if radiant not in five_man_mean:
        five_man_mean[radiant] = {'wins':0, 'duration':0, 'matches': 0, 'score_mean':0, 'deaths_mean':0, 
                                               'assists':0, 'last_hits':0, 'denies':0, 'gold_per_min':0,
                                               'xp_per_min':0}
    if dire not in five_man_mean:
        five_man_mean[dire] = {'wins':0, 'duration':0, 'matches': 0, 'score_mean':0, 'deaths_mean':0, 
                                               'assists':0, 'last_hits':0, 'denies':0, 'gold_per_min':0,
                                               'xp_per_min':0}
    
    five_man_mean[radiant]['matches'] += 1
    five_man_mean[radiant]['wins'] += r_wins
    five_man_mean[radiant]['score_mean'] += match['radiant_score']
    five_man_mean[radiant]['deaths_mean'] += match['dire_score']
    five_man_mean[radiant]['assists'] += r_assists
    five_man_mean[radiant]['last_hits'] += r_last_hits
    five_man_mean[radiant]['denies'] += r_denies
    five_man_mean[radiant]['gold_per_min'] += r_gpm
    five_man_mean[radiant]['xp_per_min'] += r_xpm
    five_man_mean[radiant]['duration'] += match['duration']
            
        
    five_man_mean[dire]['matches'] += 1
    five_man_mean[dire]['wins'] += d_wins
    five_man_mean[dire]['score_mean'] += match['dire_score']
    five_man_mean[dire]['deaths_mean'] += match['radiant_score']
    five_man_mean[dire]['assists'] += d_assists
    five_man_mean[dire]['last_hits'] += d_last_hits
    five_man_mean[dire]['denies'] += d_denies
    five_man_mean[dire]['gold_per_min'] += d_gpm
    five_man_mean[dire]['xp_per_min'] += d_xpm
    five_man_mean[dire]['duration'] += match['duration']
for five in five_man_mean:
    for stat in five_man_mean[five]:
        if stat != 'matches':
            five_man_mean[five][stat] /= five_man_mean[five]['matches']
            five_man_mean[five][stat] = round(five_man_mean[five][stat], 2)
with open('../duration_info/five_man_mean.json', 'w') as file:
    json.dump(five_man_mean, file)

In [57]:
data = []
for match in matches_relevante:
    game = {}
    game['duration'] = match['duration']
    game['r_org'] = match['radiant_team_id']
    game['d_org'] = match['dire_team_id']
    game['r_win'] = match['radiant_win']

    accounts_id = {}
    for i, player in enumerate(match['players']):
        accounts_id[player['hero_id']] = player['account_id']


    r, d = 0, 0
    for pick in match['picks_bans']:

        if pick['is_pick'] == True and pick['team']==0:
            r += 1
            key = f'r_{r}'
            hero_id = f'r_{r}_player_id'
            game[key] = pick['hero_id']
            game[hero_id] = accounts_id[pick['hero_id']]

        if pick['is_pick'] == True and pick['team']==1:
            d += 1
            key = f'd_{d}'
            hero_id = f'd_{d}_player_id'
            game[key] = pick['hero_id']
            game[hero_id] = accounts_id[pick['hero_id']]

    data.append(game)

In [58]:
data = pd.DataFrame(data, columns=list(game.keys()))

In [59]:
with open('../duration_info/orgs_mean.json') as file:
    orgs_mean = json.load(file)
    
data['r_org_mean_duration'] = 0
data['d_org_mean_duration'] = 0
data['r_org_mean_score'] = 0
data['d_org_mean_score'] = 0
data['r_org_mean_deaths'] = 0
data['d_org_mean_deaths'] = 0
data['r_org_mean_assists'] = 0
data['d_org_mean_assists'] = 0
data['r_org_mean_last_hits'] = 0
data['d_org_mean_last_hits'] = 0
data['r_org_mean_denies'] = 0
data['d_org_mean_denies'] = 0
data['r_org_mean_gpm'] = 0
data['d_org_mean_gpm'] = 0
data['r_org_mean_xpm'] = 0
data['d_org_mean_xpm'] = 0
data['r_org_mean_winrate'] = 0
data['d_org_mean_winrate'] = 0

data['org_vs_org_mean_winrate'] = 0
data['org_vs_org_mean_duration'] = 0
data['org_vs_org_mean_score'] = 0
data['org_vs_org_mean_deaths'] = 0

data['r_org_vs_org_mean_assists'] = 0
data['r_org_vs_org_mean_last_hits'] = 0
data['r_org_vs_org_mean_denies'] = 0
data['r_org_vs_org_mean_gpm'] = 0
data['r_org_vs_org_mean_xpm'] = 0

data['d_org_vs_org_mean_assists'] = 0
data['d_org_vs_org_mean_last_hits'] = 0
data['d_org_vs_org_mean_denies'] = 0
data['d_org_vs_org_mean_gpm'] = 0
data['d_org_vs_org_mean_xpm'] = 0

for org in tqdm(orgs_mean):
    data.loc[data['r_org']==int(org), 'r_org_mean_winrate'] =  orgs_mean[org]['wins']
    data.loc[data['d_org']==int(org), 'd_org_mean_winrate'] =  orgs_mean[org]['wins']
    
    data.loc[data['r_org']==int(org), 'r_org_mean_duration'] =  orgs_mean[org]['duration']
    data.loc[data['d_org']==int(org), 'd_org_mean_duration'] =  orgs_mean[org]['duration']
    
    data.loc[data['r_org']==int(org), 'r_org_mean_score'] =  orgs_mean[org]['score_mean']
    data.loc[data['d_org']==int(org), 'd_org_mean_score'] =  orgs_mean[org]['score_mean']
    
    data.loc[data['r_org']==int(org), 'r_org_mean_deaths'] =  orgs_mean[org]['deaths_mean']
    data.loc[data['d_org']==int(org), 'd_org_mean_deaths'] =  orgs_mean[org]['deaths_mean']
    
    data.loc[data['r_org']==int(org), 'r_org_mean_assists'] =  orgs_mean[org]['assists']
    data.loc[data['d_org']==int(org), 'd_org_mean_assists'] =  orgs_mean[org]['assists']
    
    data.loc[data['r_org']==int(org), 'r_org_mean_last_hits'] =  orgs_mean[org]['last_hits']
    data.loc[data['d_org']==int(org), 'd_org_mean_last_hits'] =  orgs_mean[org]['last_hits']
    
    data.loc[data['r_org']==int(org), 'r_org_mean_denies'] =  orgs_mean[org]['denies']
    data.loc[data['d_org']==int(org), 'd_org_mean_denies'] =  orgs_mean[org]['denies']
    
    data.loc[data['r_org']==int(org), 'r_org_mean_gpm'] =  orgs_mean[org]['gold_per_min']
    data.loc[data['d_org']==int(org), 'd_org_mean_gpm'] =  orgs_mean[org]['gold_per_min']
    
    data.loc[data['r_org']==int(org), 'r_org_mean_xpm'] =  orgs_mean[org]['xp_per_min']
    data.loc[data['d_org']==int(org), 'd_org_mean_xpm'] =  orgs_mean[org]['xp_per_min']
    
    for versus in orgs_mean[org]['versus_orgs']:
        data.loc[(data['r_org']==int(org)) & (data['d_org']==int(versus)),'org_vs_org_mean_duration'] = orgs_mean[org]['versus_orgs'][versus]['duration']
        data.loc[(data['r_org']==int(versus)) & (data['d_org']==int(org)),'org_vs_org_mean_duration'] = orgs_mean[org]['versus_orgs'][versus]['duration']
        
        data.loc[(data['r_org']==int(org)) & (data['d_org']==int(versus)),'org_vs_org_mean_winrate'] = orgs_mean[org]['versus_orgs'][versus]['wins']
        data.loc[(data['r_org']==int(versus)) & (data['d_org']==int(org)),'org_vs_org_mean_winrate'] = orgs_mean[org]['versus_orgs'][versus]['wins']
        
        data.loc[(data['r_org']==int(org)) & (data['d_org']==int(versus)),'org_vs_org_mean_score'] = orgs_mean[org]['versus_orgs'][versus]['score_mean']
        data.loc[(data['r_org']==int(versus)) & (data['d_org']==int(org)),'org_vs_org_mean_score'] = orgs_mean[org]['versus_orgs'][versus]['deaths_mean']
        
        data.loc[(data['r_org']==int(org)) & (data['d_org']==int(versus)),'org_vs_org_mean_deaths'] = orgs_mean[versus]['versus_orgs'][org]['score_mean']
        data.loc[(data['r_org']==int(versus)) & (data['d_org']==int(org)),'org_vs_org_mean_deaths'] = orgs_mean[versus]['versus_orgs'][org]['deaths_mean']
        
        data.loc[(data['r_org']==int(org)) & (data['d_org']==int(versus)),'r_org_vs_org_mean_assists'] = orgs_mean[org]['versus_orgs'][versus]['r_assists']
        data.loc[(data['r_org']==int(org)) & (data['d_org']==int(versus)),'d_org_vs_org_mean_assists'] = orgs_mean[org]['versus_orgs'][versus]['d_assists']
        data.loc[(data['r_org']==int(versus)) & (data['d_org']==int(org)),'r_org_vs_org_mean_assists'] = orgs_mean[versus]['versus_orgs'][org]['r_assists']
        data.loc[(data['r_org']==int(versus)) & (data['d_org']==int(org)),'d_org_vs_org_mean_assists'] = orgs_mean[versus]['versus_orgs'][org]['d_assists']
        
        data.loc[(data['r_org']==int(org)) & (data['d_org']==int(versus)),'r_org_vs_org_mean_last_hits'] = orgs_mean[org]['versus_orgs'][versus]['r_last_hits']
        data.loc[(data['r_org']==int(org)) & (data['d_org']==int(versus)),'d_org_vs_org_mean_last_hits'] = orgs_mean[org]['versus_orgs'][versus]['d_last_hits']
        data.loc[(data['r_org']==int(versus)) & (data['d_org']==int(org)),'r_org_vs_org_mean_last_hits'] = orgs_mean[versus]['versus_orgs'][org]['r_last_hits']
        data.loc[(data['r_org']==int(versus)) & (data['d_org']==int(org)),'d_org_vs_org_mean_last_hits'] = orgs_mean[versus]['versus_orgs'][org]['d_last_hits']
        
        data.loc[(data['r_org']==int(org)) & (data['d_org']==int(versus)),'r_org_vs_org_mean_denies'] = orgs_mean[org]['versus_orgs'][versus]['r_denies']
        data.loc[(data['r_org']==int(org)) & (data['d_org']==int(versus)),'d_org_vs_org_mean_denies'] = orgs_mean[org]['versus_orgs'][versus]['d_denies']
        data.loc[(data['r_org']==int(versus)) & (data['d_org']==int(org)),'r_org_vs_org_mean_denies'] = orgs_mean[versus]['versus_orgs'][org]['r_denies']
        data.loc[(data['r_org']==int(versus)) & (data['d_org']==int(org)),'d_org_vs_org_mean_denies'] = orgs_mean[versus]['versus_orgs'][org]['d_denies']
        
        data.loc[(data['r_org']==int(org)) & (data['d_org']==int(versus)),'r_org_vs_org_mean_gpm'] = orgs_mean[org]['versus_orgs'][versus]['r_gold_per_min']
        data.loc[(data['r_org']==int(org)) & (data['d_org']==int(versus)),'d_org_vs_org_mean_gpm'] = orgs_mean[org]['versus_orgs'][versus]['d_gold_per_min']
        data.loc[(data['r_org']==int(versus)) & (data['d_org']==int(org)),'r_org_vs_org_mean_gpm'] = orgs_mean[versus]['versus_orgs'][org]['r_gold_per_min']
        data.loc[(data['r_org']==int(versus)) & (data['d_org']==int(org)),'d_org_vs_org_mean_gpm'] = orgs_mean[versus]['versus_orgs'][org]['d_gold_per_min']
        
        data.loc[(data['r_org']==int(org)) & (data['d_org']==int(versus)),'r_org_vs_org_mean_xpm'] = orgs_mean[org]['versus_orgs'][versus]['r_xp_per_min']
        data.loc[(data['r_org']==int(org)) & (data['d_org']==int(versus)),'d_org_vs_org_mean_xpm'] = orgs_mean[org]['versus_orgs'][versus]['d_xp_per_min']
        data.loc[(data['r_org']==int(versus)) & (data['d_org']==int(org)),'r_org_vs_org_mean_xpm'] = orgs_mean[versus]['versus_orgs'][org]['r_xp_per_min']
        data.loc[(data['r_org']==int(versus)) & (data['d_org']==int(org)),'d_org_vs_org_mean_xpm'] = orgs_mean[versus]['versus_orgs'][org]['d_xp_per_min']
        
        

100%|██████████| 368/368 [04:18<00:00,  1.42it/s]


In [60]:
with open('../duration_info/players_mean.json') as file:
    players_mean = json.load(file)
data['r_players_mean_duration'] = 0
data['d_players_mean_duration'] = 0
data['r_players_heroes_mean_duration'] = 0
data['d_players_heroes_mean_duration'] = 0

data['r_players_mean_winrate'] = 0
data['d_players_mean_winrate'] = 0
data['r_players_heroes_mean_winrate'] = 0
data['d_players_heroes_mean_winrate'] = 0

data['r_players_mean_score'] = 0
data['d_players_mean_score'] = 0
data['r_players_heroes_mean_score'] = 0
data['d_players_heroes_mean_score'] = 0

data['r_players_mean_deaths'] = 0
data['d_players_mean_deaths'] = 0
data['r_players_heroes_mean_deaths'] = 0
data['d_players_heroes_mean_deaths'] = 0

data['r_players_mean_assists'] = 0
data['d_players_mean_assists'] = 0
data['r_players_heroes_mean_assists'] = 0
data['d_players_heroes_mean_assists'] = 0

data['r_players_mean_last_hits'] = 0
data['d_players_mean_last_hits'] = 0
data['r_players_heroes_mean_last_hits'] = 0
data['d_players_heroes_mean_last_hits'] = 0

data['r_players_mean_denies'] = 0
data['d_players_mean_denies'] = 0
data['r_players_heroes_mean_denies'] = 0
data['d_players_heroes_mean_denies'] = 0

data['r_players_mean_gpm'] = 0
data['d_players_mean_gpm'] = 0
data['r_players_heroes_mean_gpm'] = 0
data['d_players_heroes_mean_gpm'] = 0

data['r_players_mean_xpm'] = 0
data['d_players_mean_xpm'] = 0
data['r_players_heroes_mean_xpm'] = 0
data['d_players_heroes_mean_xpm'] = 0

for player in tqdm(players_mean):
    for i in range(1, 6):
        r_player = f'r_{i}_player_id'
        d_player = f'd_{i}_player_id'
        
        data.loc[data[r_player] == int(player), 'r_players_mean_duration'] += players_mean[player]['duration']
        data.loc[data[d_player] == int(player), 'd_players_mean_duration'] += players_mean[player]['duration']
        
        data.loc[data[r_player] == int(player), 'r_players_mean_winrate'] += players_mean[player]['wins']
        data.loc[data[d_player] == int(player), 'd_players_mean_winrate'] += players_mean[player]['wins']
        
        data.loc[data[r_player] == int(player), 'r_players_mean_score'] += players_mean[player]['score_mean']
        data.loc[data[d_player] == int(player), 'd_players_mean_score'] += players_mean[player]['score_mean']
        
        data.loc[data[r_player] == int(player), 'r_players_mean_deaths'] += players_mean[player]['deaths_mean']
        data.loc[data[d_player] == int(player), 'd_players_mean_deaths'] += players_mean[player]['deaths_mean']
        
        data.loc[data[r_player] == int(player), 'r_players_mean_assists'] += players_mean[player]['assists']
        data.loc[data[d_player] == int(player), 'd_players_mean_assists'] += players_mean[player]['assists']
        
        data.loc[data[r_player] == int(player), 'r_players_mean_last_hits'] += players_mean[player]['last_hits']
        data.loc[data[d_player] == int(player), 'd_players_mean_last_hits'] += players_mean[player]['last_hits']
        
        data.loc[data[r_player] == int(player), 'r_players_mean_denies'] += players_mean[player]['denies']
        data.loc[data[d_player] == int(player), 'd_players_mean_denies'] += players_mean[player]['denies']
        
        data.loc[data[r_player] == int(player), 'r_players_mean_gpm'] += players_mean[player]['gold_per_min']
        data.loc[data[d_player] == int(player), 'd_players_mean_gpm'] += players_mean[player]['gold_per_min']
        
        data.loc[data[r_player] == int(player), 'r_players_mean_xpm'] += players_mean[player]['xp_per_min']
        data.loc[data[d_player] == int(player), 'd_players_mean_xpm'] += players_mean[player]['xp_per_min']
    for hero in players_mean[player]['heroes']:
        for i in range(1, 6):
            r_hero = f'r_{i}'
            d_hero = f'd_{i}'
            r_player = f'r_{i}_player_id'
            d_player = f'd_{i}_player_id'
            
            data.loc[(data[r_player] == int(player)) & (data[r_hero] == int(hero)), 'r_players_heroes_mean_duration'] += players_mean[player]['heroes'][hero]['duration']
            data.loc[(data[d_player] == int(player)) & (data[d_hero] == int(hero)), 'd_players_heroes_mean_duration'] += players_mean[player]['heroes'][hero]['duration']
            
            
            data.loc[(data[r_player] == int(player)) & (data[r_hero] == int(hero)), 'r_players_heroes_mean_winrate'] += players_mean[player]['heroes'][hero]['wins']
            data.loc[(data[d_player] == int(player)) & (data[d_hero] == int(hero)), 'd_players_heroes_mean_winrate'] += players_mean[player]['heroes'][hero]['wins']
            
            data.loc[(data[r_player] == int(player)) & (data[r_hero] == int(hero)), 'r_players_heroes_mean_score'] += players_mean[player]['heroes'][hero]['score_mean']
            data.loc[(data[d_player] == int(player)) & (data[d_hero] == int(hero)), 'd_players_heroes_mean_score'] += players_mean[player]['heroes'][hero]['score_mean']
            
            data.loc[(data[r_player] == int(player)) & (data[r_hero] == int(hero)), 'r_players_heroes_mean_deaths'] += players_mean[player]['heroes'][hero]['deaths_mean']
            data.loc[(data[d_player] == int(player)) & (data[d_hero] == int(hero)), 'd_players_heroes_mean_deaths'] += players_mean[player]['heroes'][hero]['deaths_mean']
            
            data.loc[(data[r_player] == int(player)) & (data[r_hero] == int(hero)), 'r_players_heroes_mean_assists'] += players_mean[player]['heroes'][hero]['assists']
            data.loc[(data[d_player] == int(player)) & (data[d_hero] == int(hero)), 'd_players_heroes_mean_assists'] += players_mean[player]['heroes'][hero]['assists']
            
            data.loc[(data[r_player] == int(player)) & (data[r_hero] == int(hero)), 'r_players_heroes_mean_last_hits'] += players_mean[player]['heroes'][hero]['last_hits']
            data.loc[(data[d_player] == int(player)) & (data[d_hero] == int(hero)), 'd_players_heroes_mean_last_hits'] += players_mean[player]['heroes'][hero]['last_hits']
            
            data.loc[(data[r_player] == int(player)) & (data[r_hero] == int(hero)), 'r_players_heroes_mean_denies'] += players_mean[player]['heroes'][hero]['denies']
            data.loc[(data[d_player] == int(player)) & (data[d_hero] == int(hero)), 'd_players_heroes_mean_denies'] += players_mean[player]['heroes'][hero]['denies']
            
            data.loc[(data[r_player] == int(player)) & (data[r_hero] == int(hero)), 'r_players_heroes_mean_gpm'] += players_mean[player]['heroes'][hero]['gold_per_min']
            data.loc[(data[d_player] == int(player)) & (data[d_hero] == int(hero)), 'd_players_heroes_mean_gpm'] += players_mean[player]['heroes'][hero]['gold_per_min']
            
            data.loc[(data[r_player] == int(player)) & (data[r_hero] == int(hero)), 'r_players_heroes_mean_xpm'] += players_mean[player]['heroes'][hero]['xp_per_min']
            data.loc[(data[d_player] == int(player)) & (data[d_hero] == int(hero)), 'd_players_heroes_mean_xpm'] += players_mean[player]['heroes'][hero]['xp_per_min']
            
            
            
    
    
data['r_players_mean_duration'] /= 5
data['d_players_mean_duration'] /= 5
data['r_players_heroes_mean_duration'] /= 5
data['d_players_heroes_mean_duration'] /= 5

data['r_players_mean_score'] /= 5
data['d_players_mean_score'] /= 5
data['r_players_heroes_mean_score'] /= 5
data['d_players_heroes_mean_score'] /= 5

data['r_players_mean_deaths'] /= 5
data['d_players_mean_deaths'] /= 5
data['r_players_heroes_mean_deaths'] /= 5
data['d_players_heroes_mean_deaths'] /= 5

data['r_players_mean_assists'] /= 5
data['d_players_mean_assists'] /= 5
data['r_players_heroes_mean_assists'] /= 5
data['d_players_heroes_mean_assists'] /= 5

data['r_players_mean_last_hits'] /= 5
data['d_players_mean_last_hits'] /= 5
data['r_players_heroes_mean_last_hits'] /= 5
data['d_players_heroes_mean_last_hits'] /= 5

data['r_players_mean_denies'] /= 5
data['d_players_mean_denies'] /= 5
data['r_players_heroes_mean_denies'] /= 5
data['d_players_heroes_mean_denies'] /= 5

data['r_players_mean_gpm'] /= 5
data['d_players_mean_gpm'] /= 5
data['r_players_heroes_mean_gpm'] /= 5
data['d_players_heroes_mean_gpm'] /= 5

data['r_players_mean_xpm'] /= 5
data['d_players_mean_xpm'] /= 5
data['r_players_heroes_mean_xpm'] /= 5
data['d_players_heroes_mean_xpm'] /= 5

100%|██████████| 1161/1161 [1:31:50<00:00,  4.75s/it] 


In [61]:
data['r_players_mean_winrate'] /= 5
data['d_players_mean_winrate'] /= 5
data['r_players_heroes_mean_winrate'] /= 5
data['d_players_heroes_mean_winrate'] /= 5

In [62]:
with open('../duration_info/heroes_mean.json') as file:
    heroes_mean = json.load(file)
    
data['r_hero_mean_duration'] = 0
data['d_hero_mean_duration'] = 0

data['r_hero_mean_winrate'] = 0
data['d_hero_mean_winrate'] = 0

data['r_hero_mean_score'] = 0
data['d_hero_mean_score'] = 0

data['r_hero_mean_deaths'] = 0
data['d_hero_mean_deaths'] = 0

data['r_hero_mean_assists'] = 0
data['d_hero_mean_assists'] = 0

data['r_hero_mean_last_hits'] = 0
data['d_hero_mean_last_hits'] = 0

data['r_hero_mean_denies'] = 0
data['d_hero_mean_denies'] = 0

data['r_hero_mean_gpm'] = 0
data['d_hero_mean_gpm'] = 0

data['r_hero_mean_xpm'] = 0
data['d_hero_mean_xpm'] = 0
for hero in heroes_mean:
    for i in range(1, 6):
        r_hero = f'r_{i}'
        d_hero = f'd_{i}'
        
        data.loc[data[r_hero] == int(hero), 'r_hero_mean_duration'] += heroes_mean[hero]['duration']
        data.loc[data[d_hero] == int(hero), 'd_hero_mean_duration'] += heroes_mean[hero]['duration']
        
        data.loc[data[r_hero] == int(hero), 'r_hero_mean_winrate'] += heroes_mean[hero]['wins']
        data.loc[data[d_hero] == int(hero), 'd_hero_mean_winrate'] += heroes_mean[hero]['wins']
        
        
        data.loc[data[r_hero] == int(hero), 'r_hero_mean_score'] += heroes_mean[hero]['score_mean']
        data.loc[data[d_hero] == int(hero), 'd_hero_mean_score'] += heroes_mean[hero]['score_mean']
        
        data.loc[data[r_hero] == int(hero), 'r_hero_mean_deaths'] += heroes_mean[hero]['deaths_mean']
        data.loc[data[d_hero] == int(hero), 'd_hero_mean_deaths'] += heroes_mean[hero]['deaths_mean']
        
        data.loc[data[r_hero] == int(hero), 'r_hero_mean_assists'] += heroes_mean[hero]['assists']
        data.loc[data[d_hero] == int(hero), 'd_hero_mean_assists'] += heroes_mean[hero]['assists']
        
        data.loc[data[r_hero] == int(hero), 'r_hero_mean_last_hits'] += heroes_mean[hero]['last_hits']
        data.loc[data[d_hero] == int(hero), 'd_hero_mean_last_hits'] += heroes_mean[hero]['last_hits']
        
        data.loc[data[r_hero] == int(hero), 'r_hero_mean_denies'] += heroes_mean[hero]['denies']
        data.loc[data[d_hero] == int(hero), 'd_hero_mean_denies'] += heroes_mean[hero]['denies']
        
        data.loc[data[r_hero] == int(hero), 'r_hero_mean_gpm'] += heroes_mean[hero]['gold_per_min']
        data.loc[data[d_hero] == int(hero), 'd_hero_mean_gpm'] += heroes_mean[hero]['gold_per_min']
        
        data.loc[data[r_hero] == int(hero), 'r_hero_mean_xpm'] += heroes_mean[hero]['xp_per_min']
        data.loc[data[d_hero] == int(hero), 'd_hero_mean_xpm'] += heroes_mean[hero]['xp_per_min']
        
        
data['r_hero_mean_duration'] /= 5
data['d_hero_mean_duration'] /= 5

data['r_hero_mean_winrate'] /= 5
data['d_hero_mean_winrate'] /= 5

data['r_hero_mean_score'] /= 5
data['d_hero_mean_score'] /= 5

data['r_hero_mean_deaths'] /= 5
data['d_hero_mean_deaths'] /= 5

data['r_hero_mean_assists'] /= 5
data['d_hero_mean_assists'] /= 5

data['r_hero_mean_last_hits'] /= 5
data['d_hero_mean_last_hits'] /= 5

data['r_hero_mean_denies'] /= 5
data['d_hero_mean_denies'] /= 5

data['r_hero_mean_gpm'] /= 5
data['d_hero_mean_gpm'] /= 5

data['r_hero_mean_xpm'] /= 5
data['d_hero_mean_xpm'] /= 5

In [63]:
data['r_five_players_id'] = 0
data['d_five_players_id'] = 0
for i in range(data.shape[0]):
    radiant = []
    dire = []
    for j in range(1, 6):
        r = f'r_{j}_player_id'
        d = f'd_{j}_player_id'
        radiant.append(data.loc[i, r])
        dire.append(data.loc[i, d])
    radiant = ''.join(map(str, sorted(radiant)))
    dire = ''.join(map(str, sorted(dire)))
    data.loc[i, 'r_five_players_id'] = radiant
    data.loc[i, 'd_five_players_id'] = dire

In [64]:
with open('../duration_info/five_man_mean.json') as file:
    five_man_mean = json.load(file)
data['r_five_mean_duration'] = 0
data['d_five_mean_duration'] = 0

data['r_five_mean_winrate'] = 0
data['d_five_mean_winrate'] = 0

data['r_five_mean_score'] = 0
data['d_five_mean_score'] = 0

data['r_five_mean_deaths'] = 0
data['d_five_mean_deaths'] = 0

data['r_five_mean_assists'] = 0
data['d_five_mean_assists'] = 0

data['r_five_mean_last_hits'] = 0
data['d_five_mean_last_hits'] = 0

data['r_five_mean_denies'] = 0
data['d_five_mean_denies'] = 0

data['r_five_mean_gpm'] = 0
data['d_five_mean_gpm'] = 0

data['r_five_mean_xpm'] = 0
data['d_five_mean_xpm'] = 0

for five in five_man_mean:

    data.loc[data['r_five_players_id'] == five, 'r_five_mean_duration'] = five_man_mean[five]['duration']
    data.loc[data['d_five_players_id'] == five, 'd_five_mean_duration'] = five_man_mean[five]['duration']
    
    data.loc[data['r_five_players_id'] == five, 'r_five_mean_winrate'] = five_man_mean[five]['wins']
    data.loc[data['d_five_players_id'] == five, 'd_five_mean_winrate'] = five_man_mean[five]['wins']

    data.loc[data['r_five_players_id'] == five, 'r_five_mean_score'] = five_man_mean[five]['score_mean']
    data.loc[data['d_five_players_id'] == five, 'd_five_mean_score'] = five_man_mean[five]['score_mean']

    data.loc[data['r_five_players_id'] == five, 'r_five_mean_deaths'] = five_man_mean[five]['deaths_mean']
    data.loc[data['d_five_players_id'] == five, 'd_five_mean_deaths'] = five_man_mean[five]['deaths_mean']

    data.loc[data['r_five_players_id'] == five, 'r_five_mean_assists'] = five_man_mean[five]['assists']
    data.loc[data['d_five_players_id'] == five, 'd_five_mean_assists'] = five_man_mean[five]['assists']

    data.loc[data['r_five_players_id'] == five, 'r_five_mean_last_hits'] = five_man_mean[five]['last_hits']
    data.loc[data['d_five_players_id'] == five, 'd_five_mean_last_hits'] = five_man_mean[five]['last_hits']

    data.loc[data['r_five_players_id'] == five, 'r_five_mean_denies'] = five_man_mean[five]['denies']
    data.loc[data['d_five_players_id'] == five, 'd_five_mean_denies'] = five_man_mean[five]['denies']

    data.loc[data['r_five_players_id'] == five, 'r_five_mean_gpm'] = five_man_mean[five]['gold_per_min']
    data.loc[data['d_five_players_id'] == five, 'd_five_mean_gpm'] = five_man_mean[five]['gold_per_min']

    data.loc[data['r_five_players_id'] == five, 'r_five_mean_xpm'] = five_man_mean[five]['xp_per_min']
    data.loc[data['d_five_players_id'] == five, 'd_five_mean_xpm'] = five_man_mean[five]['xp_per_min']
        
        
'''data['r_five_mean_duration'] /= 5
data['d_five_mean_duration'] /= 5

data['r_five_mean_score'] /= 5
data['d_five_mean_score'] /= 5

data['r_five_mean_deaths'] /= 5
data['d_five_mean_deaths'] /= 5

data['r_five_mean_assists'] /= 5
data['d_five_mean_assists'] /= 5

data['r_five_mean_last_hits'] /= 5
data['d_five_mean_last_hits'] /= 5

data['r_five_mean_denies'] /= 5
data['d_five_mean_denies'] /= 5

data['r_five_mean_gpm'] /= 5
data['d_five_mean_gpm'] /= 5

data['r_five_mean_xpm'] /= 5
data['d_five_mean_xpm'] /= 5'''

"data['r_five_mean_duration'] /= 5\ndata['d_five_mean_duration'] /= 5\n\ndata['r_five_mean_score'] /= 5\ndata['d_five_mean_score'] /= 5\n\ndata['r_five_mean_deaths'] /= 5\ndata['d_five_mean_deaths'] /= 5\n\ndata['r_five_mean_assists'] /= 5\ndata['d_five_mean_assists'] /= 5\n\ndata['r_five_mean_last_hits'] /= 5\ndata['d_five_mean_last_hits'] /= 5\n\ndata['r_five_mean_denies'] /= 5\ndata['d_five_mean_denies'] /= 5\n\ndata['r_five_mean_gpm'] /= 5\ndata['d_five_mean_gpm'] /= 5\n\ndata['r_five_mean_xpm'] /= 5\ndata['d_five_mean_xpm'] /= 5"

In [66]:
url = 'https://api.opendota.com/api/heroStats'
heroes_stats = requests.get(url).json()

heroes = {}
for hero in heroes_stats:
    heroes[hero['hero_id']] = hero

In [67]:
def get_lvl(duration, xpm):
    duration /= 60
    xpm = xpm * duration
    lvl = 1
    while True:
        if xpm - lvl * 100 < 0:
            lvl += xpm / ((lvl+1) * 100)
            break
        else:
            xpm -= lvl * 100
        lvl += 1
    return round(lvl, 2)

In [68]:
def get_stats(heroes_id, players_id):
    stats = {}
    stats['health'] = 0
    stats['mana'] = 0
    stats['armor'] = 0
    for hero, player in zip(heroes_id, players_id):
        lvl = get_lvl(players_mean[str(player)]['heroes'][str(hero)]['duration'], players_mean[str(player)]['heroes'][str(hero)]['xp_per_min'])
        
        stats['health'] += heroes[hero]['base_health']
        stats['armor'] += heroes[hero]['base_armor']
        stats['mana'] += heroes[hero]['base_mana']
        
        strange = lvl * heroes[hero]['str_gain'] + heroes[hero]['base_str']
        agility = lvl * heroes[hero]['agi_gain'] + heroes[hero]['base_agi']
        intellect = lvl * heroes[hero]['int_gain'] + heroes[hero]['base_int']
        
        stats['health'] += strange * 20
        stats['armor'] += agility * 0.16
        stats['mana'] += intellect * 12
        
    stats['health'] /= 5
    stats['armor'] /= 5
    stats['mana'] /= 5

    stats['health'] = round(stats['health'], 2)
    stats['armor'] = round(stats['armor'], 2)
    stats['mana'] = round(stats['mana'], 2)
    return stats['health'], stats['armor'], stats['mana']

In [69]:
data['r_heroes_players_mean_health'] = 0
data['d_heroes_players_mean_health'] = 0

data['r_heroes_players_mean_armor'] = 0
data['d_heroes_players_mean_armor'] = 0

data['r_heroes_players_mean_mana'] = 0
data['d_heroes_players_mean_mana'] = 0
for index, i in tqdm(enumerate(data.index)):
    r_hero_id = list(data.loc[i, [f'r_{i}' for i in range(1, 6)]].values)
    d_hero_id = list(data.loc[i, [f'd_{i}' for i in range(1, 6)]].values)

    r_players_id = list(data.loc[i, [f'r_{i}_player_id' for i in range(1, 6)]].values)
    d_players_id = list(data.loc[i, [f'd_{i}_player_id' for i in range(1, 6)]].values)

    r_stats = get_stats(r_hero_id, r_players_id)
    d_stats = get_stats(d_hero_id, d_players_id)

    data.loc[i, 'r_heroes_players_mean_health'] = r_stats[0]
    data.loc[i, 'd_heroes_players_mean_health'] = d_stats[0]

    data.loc[i, 'r_heroes_players_mean_armor'] = r_stats[1]
    data.loc[i, 'd_heroes_players_mean_armor'] = d_stats[1]

    data.loc[i, 'r_heroes_players_mean_mana'] = r_stats[2]
    data.loc[i, 'd_heroes_players_mean_mana'] = d_stats[2]

10656it [01:30, 118.31it/s]


In [70]:
data.to_csv('train.csv', encoding='utf-8', index=False)